In [16]:
from nltk.tokenize import sent_tokenize, word_tokenize
import random
import nltk
from operator import itemgetter
import re
import numpy as np

In [4]:
def addtG(TOKEN):
    
    for i in range (len(TOKEN)):
        TOKEN[i] = re.sub(r'^',"<s>",TOKEN[i])
        TOKEN[i] = re.sub(r'$',"</s>",TOKEN[i])
    #print(TOKEN)


In [5]:
#unigram MLE
def uniP(freq):
    numTok = len(unigrams)
    prob = {}
    for (key,fvalue) in reversed(sorted(freq.items(), key = itemgetter(1))):
        prob[key] = fvalue/numTok
    
    return prob


In [6]:
def textProc(file_name):
    f= open(file_name,'r')
    #tp1
    corpus = f.read().replace('\n',' ') #replace the new line with single white space.
    corpus = corpus.lower() #tp2

    token = sent_tokenize(corpus)

    #replace all the non alphanumeric characters with whitespace.
    token = [re.sub(r'[^A-Za-z0-9]'," ", i) for i in token]#tp3

    random.shuffle(token)

    #addtG(token) #add tags
    return token

    #clean tokens of sentence returned


In [7]:
#write a function for count of n-gram
def LstOfNgram(count,corpus):
    #cleaned corpus
    n_gram=[]
    for i in corpus:
        p = word_tokenize(i)
        p = ["<s>"]+p
        p.append("</s>")    
        #token must have start and end line tags here.
        #print (p)
        for g in range(count-1,len(p)):
            dummy=""
            for j in range(count):
                dummy = p[g-j]+" "+dummy   #i,i-1,i-2,i-3,i-4,i-5,...,i-(n-1)
               #more things to be added here
            dummy = re.sub(r"^\s|\s$","",dummy)
            n_gram.append(dummy)
    return (n_gram)    

    

In [8]:
def CountNgram(frequency):
    count ={}
    
    for (key,fvalue) in reversed(sorted(frequency.items(), key = itemgetter(1))):
        count[key] = fvalue
 
    return count

In [41]:
#unified method for MLE of n-gram
def MLE(a,b,N):
    if N ==1:
        return (count_unigram[a]/len(unigramLst))
    elif N==2:
        return (count_bigram[b+" "+a]/count_unigram[b])
    elif N == 3:
        return (count_trigram[b+" "+a]/count_bigram[b])
    elif N == 4:
        return (count_quadgram[b+" "+a]/count_trigram[b])

In [25]:
#question 5
def add_1_prob(a,b):#specificaly for bigrams
    try:
        numerator = count_bigram[b+" "+a] + 1
    except :
        numerator = 1
    try:   
        denominator = count_unigram[b] + len(count_unigram) #count_unigram is count of unigram(types in corpus)
    except :
        denominator = len(count_unigram)
    return (numerator/denominator)
#assure once that it is len(count_bigam)

In [12]:
def get_NLst(count_dict):#pass dicationary of word vs count.
    #create counting list for 
    l = count_dict 
    l=list(l.values())
    l.sort()
    #we want a dictionary with {1:1,2:1,3:2,4:3,5:1}
    d={}
    for i in range(len(l)):
        count =1
        for j in range(i,0,-1):
            if l[i] == l[j-1]:
                count = count + 1
            else:
                break 
    
        d[l[i]] = count
    return (d)
#takes O(n^2) which takes lot of time to execute.
    

In [64]:
def good_turing(count):#do we have to compute this for all the unigrams, bigrams, trigrams and quadgrams?
    #for bigrams
   
    dictN = get_NLst(count)#dictionary of all the N's and corresponding values
    keys = list(dictN.keys())
    print (dictN)
    values = list(dictN.values())
    l=count.copy()
    for i in l.keys():
        try:
            l[i] = (l[i]+1)*(dictN[l[i]+1]/dictN[l[i]])
        except:
            intr = np.interp(l[i]+1,keys,values)
            print ("the interpolated value for %d is %d",l[i]+1,intr)
            l[i] = (l[i]+1)*(intr/dictN[l[i]])

    return l    
 #this function just adds one to the existing count, what is the issue?       
        
   
        

In [ ]:
def good_tunning_bigram(old_count_bigram):
    dictN = get_NLst(old_count_bigram)
    
    

In [13]:

token = textProc('SherlockHomes.txt')


train = token[:int(len(token)*0.8)]#80%
validation = token[int(len(token)*0.8):]#20%
#print (token)


In [14]:
unigrams = []
for i in token:#token is token of sentence.
    p=word_tokenize(i)
    p = ["<s>"]+p
    p.append("</s>")
    unigrams+=p
#HOW TO GET RIDGE OF THIS 
#print (unigrams)
#MLE of unigram
#frequency of occurence divided by total number of tokens
dict_freq = nltk.FreqDist(unigrams)
MLE_unigram = uniP(dict_freq)

#print (MLE_unigram)


In [23]:
#n-gram count
#token = ["this is a sentence","this is something differnt","this is something differnt that this has happend"]
#list contining n-gram
unigramLst = LstOfNgram(1,token)  ##the first number here representes the N of N-gram.
bigramLst  = LstOfNgram(2,token)
trigramLst = LstOfNgram(3,token)
quadgramLst= LstOfNgram(4,token)
#print (bigramLst)
#  query for count of unigram only once
feq  =  nltk.FreqDist(unigramLst)
freq =  nltk.FreqDist(bigramLst)
fre  =  nltk.FreqDist(trigramLst)
fr   =  nltk.FreqDist(quadgramLst)

#count_unigram={}
count_unigram = CountNgram(feq)
count_bigram = CountNgram(freq)
count_trigram = CountNgram(fre)
count_quadgram = CountNgram(fr)
print (count_bigram)


In [51]:
print (count_bigram)

{'<s> i': 1015, 'of the': 708, 'in the': 505, '<s> it': 442, '<s> he': 357, 'it is': 335, '<s> the': 328, 'to the': 304, 'i have': 301, 'it was': 279, '<s> you': 272, 'that i': 250, 'at the': 237, 'and i': 217, '<s> and': 202, 'and the': 199, 'to be': 199, 'upon the': 196, 'it </s>': 191, 'i was': 186, 'with a': 184, 'i am': 182, '<s> but': 182, '<s> there': 170, 'of a': 169, 'i had': 168, 'was a': 160, 'that he': 154, 'he was': 151, 'that the': 149, 'is a': 143, 'on the': 142, 'said he': 140, 'with the': 138, 'in a': 138, '<s> then': 136, 'from the': 136, 'you have': 135, 'to me': 134, 'of his': 131, 'he had': 130, 'have been': 130, 'into the': 125, 'you </s>': 123, 'by the': 123, 'and a': 122, 'in his': 120, 'me </s>': 120, 'as i': 119, 'had been': 117, '<s> we': 117, 'that it': 116, '<s> what': 115, 'him </s>': 114, 'for the': 110, 'said holmes': 110, '<s> she': 109, 'that you': 108, 'there was': 106, 'there is': 104, 'but i': 104, 'which i': 103, '<s> yes': 102, 'i should': 99, 'a 

In [65]:
#try goods tuning
l={'<s> i': 1015, 'of the': 708, 'in the': 505, '<s> it': 442, '<s> he': 357, 'it is': 335, '<s> the': 328, 'to the': 304, 'i have': 301, 'it was': 279, '<s> you': 272, 'that i': 250, 'at the': 237, 'and i': 217, '<s> and': 202, 'and the': 199, 'to be': 199, 'upon the': 196, 'it </s>': 191, 'i was': 186, 'with a': 184, 'i am': 182, '<s> but': 182, '<s> there': 170, 'of a': 169, 'i had': 168, 'was a': 160, 'that he': 154, 'he was': 151, 'that the': 149, 'is a': 143, 'on the': 142, 'said he': 140}
g = good_turing(l)
print ("\n")
print(g)
print ("\n")
print (l)


{140: 1, 142: 1, 143: 1, 149: 1, 151: 1, 154: 1, 160: 1, 168: 1, 169: 1, 170: 1, 182: 2, 184: 1, 186: 1, 191: 1, 196: 1, 199: 2, 202: 1, 217: 1, 237: 1, 250: 1, 272: 1, 279: 1, 301: 1, 304: 1, 328: 1, 335: 1, 357: 1, 442: 1, 505: 1, 708: 1, 1015: 1}
the interpolated value for %d is %d 1016 1.0
the interpolated value for %d is %d 709 1.0
the interpolated value for %d is %d 506 1.0
the interpolated value for %d is %d 443 1.0
the interpolated value for %d is %d 358 1.0
the interpolated value for %d is %d 336 1.0
the interpolated value for %d is %d 329 1.0
the interpolated value for %d is %d 305 1.0
the interpolated value for %d is %d 302 1.0
the interpolated value for %d is %d 280 1.0
the interpolated value for %d is %d 273 1.0
the interpolated value for %d is %d 251 1.0
the interpolated value for %d is %d 238 1.0
the interpolated value for %d is %d 218 1.0
the interpolated value for %d is %d 203 1.0
the interpolated value for %d is %d 200 1.6666666666666667
the interpolated value for %d 

In [48]:
#n-gram evalutation, how many n-grams are there?
print (dictN)

{1: 36535, 2: 5948, 3: 2287, 4: 1206, 5: 690, 6: 474, 7: 305, 8: 243, 9: 202, 10: 173, 11: 138, 12: 101, 13: 94, 14: 58, 15: 49, 16: 48, 17: 44, 18: 51, 19: 41, 20: 34, 21: 27, 22: 29, 23: 22, 24: 18, 25: 15, 26: 20, 27: 15, 28: 20, 29: 15, 30: 22, 31: 13, 32: 14, 33: 12, 34: 6, 35: 9, 36: 9, 37: 15, 38: 6, 39: 9, 40: 10, 41: 6, 42: 5, 43: 8, 44: 1, 45: 6, 46: 5, 47: 4, 48: 6, 49: 2, 50: 6, 51: 3, 52: 3, 53: 7, 54: 2, 55: 1, 56: 2, 57: 3, 58: 1, 59: 2, 60: 2, 61: 5, 62: 1, 63: 3, 64: 1, 65: 3, 66: 2, 67: 1, 68: 4, 70: 3, 71: 1, 74: 4, 76: 2, 77: 2, 79: 1, 80: 2, 81: 1, 82: 2, 83: 2, 84: 2, 86: 2, 88: 1, 89: 2, 90: 1, 92: 1, 93: 1, 95: 3, 96: 1, 98: 2, 99: 3, 102: 1, 103: 1, 104: 2, 106: 1, 108: 1, 109: 1, 110: 2, 114: 1, 115: 1, 116: 1, 117: 2, 119: 1, 120: 2, 122: 1, 123: 2, 125: 1, 130: 2, 131: 1, 134: 1, 135: 1, 136: 2, 138: 2, 140: 1, 142: 1, 143: 1, 149: 1, 151: 1, 154: 1, 160: 1, 168: 1, 169: 1, 170: 1, 182: 2, 184: 1, 186: 1, 191: 1, 196: 1, 199: 2, 202: 1, 217: 1, 237: 1, 250: 

In [40]:
#question 3 and question 
f1 = MLE("a","",1)
f2 = MLE("a","is",2)#probability of bigram 
f3 = MLE("a", "this is",3)
#print(f1)
#print (f2)
#print (f3)
print (MLE("is","this",2))

#MLE_bigram("this","is") "this" given "is" = count("is this")/count(is)

#finding quadgram of a small courpus is of no use do appears to be a good idea.
#p = good_turing(count_bigram)
#p


#how to do the second part?

0.08196721311475409


In [42]:
#QUESTION 5
print (add_1_prob("is","this"))#discounted probability
print (add_1_prob("nott","here in"))#new probabilited to the one that is not present
print (add_1_prob("not","here "))#new probabilited to the one that is not present

0.004895522388059702
0.00012679092177000128
0.00012679092177000128


In [ ]:
#question 6
#goods tunning updates the count of all the biagrams

In [35]:
l1 =[ i[1] for i in p.items()]
l2 =[ j[1] for j in count_bigram.items()]
l = l1[5] - l2[5]
print (l1[0:10],l2[0:10])

[1016.0, 709.0, 506.0, 443.0, 358.0, 336.0, 329.0, 305.0, 302.0, 280.0] [1015, 708, 505, 442, 357, 335, 328, 304, 301, 279]


In [ ]:
#rough work below[a]/count_unigram

In [ ]:
l="<s>this is a sentence here is.</s>"
token = word_tokenize(l)
#print (token)
regx = re.compile("[a-z0-9]+ [a-z0-9]+")
r =re.findall(regx,l)
p= ["<s>this", "this is","is a ","a sentence","sentence here","here is","is. </s>"]
print (r)

In [ ]:



import re
l = "this is a sentence here is"
token1 = word_tokenize(l)
token1 = ["<s>"]+token1
token1.append("</s>")
n_gram = []
n=2 #bigram 
#so keeping a sentence token is a good idea
#for i in range(n-1,len(token)):#start with the n-1 of the n gram, so that it does not give any error
 #   n_gram.append(token[i-1]+" "+token[i])
#print (n_gram)

#for n gram              <s>this is a sentence here is</s>
print (token1)
for i in range(n-1,len(token1)):
    p=" "
    for j in range(n):
       p = token1[i-j]+" "+p   #i,i-1,i-2,i-3,i-4,i-5,...,i-(n-1)
       #more things to be added here
    n_gram.append(p)
#print (n_gram)

In [ ]:
1

In [ ]:
l={"this":4,"is":3,"my":4,"country":3,"hello":1,"bitch":2}



In [ ]:
l=[1,2,3,3,4,4,4,5,5,5,5,5,5,7,8,8,10,10,10,11,12]
l={"this":4,"is":3,"my":4,"country":3,"hello":1,"bitch":2}

l=list(l.values())
l.sort()

#we want a dictionary with {1:1,2:1,3:2,4:3,5:1}
d={}
for i in range(len(l)):
    count =1
    for j in range(i,0,-1):
        if l[i] == l[j-1]:
            count = count + 1
          
        else:
            break 
    
    d[l[i]] = count
d

In [ ]:
p = get_NLst()
p


In [ ]:
print (p)

In [ ]:
l=[1,2,3,4,5]
p=l.copy()
p[0]= 2
p



In [ ]:
import numpy as np
l={"this":4,"is":3,"my":4,"country":3,"hello":1,"bitch":2}

dictN = get_NLst(l)#dictionary of all the N's and corresponding values

keys = list(dictN.keys())
values = list(dictN.values())
for i in l.keys():
    try:
       
        l[i] = (l[i]+1)*(dictN[l[i]+1]/dictN[l[i]])
    except:
        intr = np.interp(l[i]+1,keys,values)
        
        l[i] = (l[i]+1)*(intr/dictN[l[i]])
l


In [ ]:
#use zip somewhere where it belongss
